# Alternatives pour prédire un métier à partir de compétences

Plusieurs approches sont possibles pour relier un profil de compétences à un métier.  
Notre choix : **représentation sparse + retrieval cosinus** (simple, efficace, interprétable).  

| Approche | Principe | Avantages | Limites |
|----------|----------|-----------|---------|
| **Classification supervisée** | Entraîner un classifieur (LR, RF, etc.) avec X=compétences, y=métier | Directe, efficace si beaucoup d’exemples | Ici impossible : 1 seul vecteur par métier |
| **Embeddings (Word2Vec, Doc2Vec)** | Représenter métiers/compétences comme vecteurs denses appris | Capture proximités fines, généralisable | Besoin de gros volumes de données |
| **Graphe biparti + embeddings** | Graphe métier ↔ compétence, diffusion de similarité | Exploite liens directs/indirects | Mise en œuvre plus lourde |
| **Similarity learning (Siamese, contrastif)** | Modèle qui apprend à rapprocher les bons couples | Très adapté au retrieval | Demande beaucoup de préparation et Deep Learning |
| **Réduction dimensionnelle (SVD, autoencoder)** | Compresser la matrice en espace latent | Rapide, réduit bruit, accélère requêtes | Moins interprétable |
| **Sparse + cosinus (choisi)** | Matrice métier × compétence + cosinus de similarité | Simple, robuste, explicable | Dépend des seuils et du filtrage |

Conclusion : vu la structure des données (1584 métiers, 16 583 compétences), la solution **sparse + cosinus** est la plus adaptée.
___

# Objectif

Prédire un **métier (code_rome)** à partir d’une **liste de compétences (code_ogr_competence)**.

## Approche
1) Chargement et préparation minimale
2) Poids et variantes de cellule
3) Construction de la matrice sparse + filtrage vocabulaire
4) Évaluations retrieval
 - **Masking** (masquage partiel)
 - **LOSO** (Leave-One-Skill-Out)
5) Run des variantes et tableau comparatif
6) Profils partiels réalistes
7) Inférence simple + seuil de confiance
8) Démo d’inférence
9) Sauvegarde et rechargement des artefacts - Interopérabilité

---
### Initialisation de l'environnement

In [ ]:
# Prerequisites:
%pip install -r requirements.txt

In [79]:
# Importation des librairies
import boto3
import io
import joblib
import numpy as np
import os
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown, display
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize


In [65]:
# Load environment variables with specific location depend on EDI
if not load_dotenv('../settings/.env'):
    print("Load env from alternative from path")
    load_dotenv('script/settings/.env')

# Check settings (for debug session only)
if __debug__:
    print('Debug ON')
    print("Environment data:", 
        "\nS3_ENDPOINT_URL:", os.getenv("S3_ENDPOINT_URL"),
        "\nS3_ACCESS_KEY_ID (len):", len(os.getenv("S3_ACCESS_KEY_ID")), "bytes",
        "\nS3_SECRET_ACCESS_KEY (len):", len(os.getenv("S3_SECRET_ACCESS_KEY")), "bytes",
        "\nS3_REGION:", os.getenv("S3_REGION")
        )

Debug ON
Environment data: 
S3_ENDPOINT_URL: https://bhckzdwrhhfaxbidmwpm.supabase.co/storage/v1/s3 
S3_ACCESS_KEY_ID (len): 32 bytes 
S3_SECRET_ACCESS_KEY (len): 64 bytes 
S3_REGION: eu-west-3


In [66]:
# Init S3 client 
try:
    s3_client = boto3.client(
        service_name='s3',
        region_name=os.getenv("S3_REGION"),
        endpoint_url=os.getenv("S3_ENDPOINT_URL"),
        aws_access_key_id=os.getenv("S3_ACCESS_KEY_ID"), 
        aws_secret_access_key=os.getenv("S3_SECRET_ACCESS_KEY")
    ) 
except Exception as ex:
    print(ex)

# Define function to read ROME CSV files
def read_ml_from_s3(s3_file_key):
    bucket_name = 'ML'
    s3_ml_path = ''

    try:
        obj = s3_client.get_object(Bucket= bucket_name, Key= s3_ml_path + s3_file_key)
        df = pd.read_csv(io.BytesIO(obj['Body'].read()))
        return df
    except Exception as ex:
        print("Erreur de lecture du fichier depuis le buecket S3:")
        print("->", ex)
    
    return None

# S3 CSV reading test
df = read_ml_from_s3( 'df_competence_rome_eda_v2.csv')
if df is not None:
    print("Lecture du fichier réussi:")
    print(f"-> CSV: Nb de lignes: {df.shape[0]}, Nb de colonnes: {df.shape[1]}")

Lecture du fichier réussi:
-> CSV: Nb de lignes: 38961, Nb de colonnes: 17


___
## 1) Chargement et préparation minimale

Objectif : partir du CSV brut et produire un DataFrame propre limité aux colonnes utiles pour la suite.

### Étapes
1. **Lecture** du CSV source.
2. **Sélection** des colonnes utiles :
   - `code_rome`, `code_ogr_competence`, `sous_cat_comp`, `coeur_metier`
3. **Nettoyage léger** :
   - Trim des espaces (`str.strip()`).
   - Remplacement des `NaN` de `coeur_metier` par `"Secondaire"`.
   - Harmonisation des chaînes `"nan"` éventuelles.
4. **Contrôles rapides** :
   - Dimensions, aperçu `head()`.
   - Comptes uniques métiers/compétences.
   - Vérif de doublons potentiels.

Résultat attendu : un `df` minimal, propre, prêt pour la pondération et la matrice sparse.


In [69]:
# Chargement & préparation minimale

# 1) Lecture CSV
file_path_name = 'df_competence_rome_eda_v2.csv'
try:
    df_raw = read_ml_from_s3( file_path_name)
    print(f"Arborescence: Nb de lignes: {df.shape[0]}, Nb de colonnes: {df.shape[1]}")
except:
    print('Erreur lors de la lecture du fichier!')

# 2) Colonnes utiles
cols_min = ["code_rome", "code_ogr_competence", "sous_cat_comp", "coeur_metier"]
missing = [c for c in cols_min if c not in df_raw.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans le CSV: {missing}")

df = df_raw[cols_min].copy()

# 3) Nettoyage léger
for c in cols_min:
    df[c] = df[c].astype(str).str.strip()

# coeur_metier: remplacer NaN / 'nan' / vides -> "Secondaire"
df["coeur_metier"] = (
    df["coeur_metier"]
      .replace({"nan": np.nan, "NaN": np.nan, "None": np.nan, "": np.nan})
      .fillna("Secondaire")
)

# 4) Contrôles rapides (brut, sans matrice X)
print("Shape df:", df.shape)
print("Métiers uniques (brut):", df["code_rome"].nunique())
print("Compétences uniques (brut):", df["code_ogr_competence"].nunique())
print("Doublons exacts (4 colonnes):", df.duplicated(subset=cols_min).sum())

print("\nTop sous_cat_comp:")
print(df["sous_cat_comp"].value_counts().head(10))

print("\nValeurs coeur_metier après imputation:")
print(df["coeur_metier"].value_counts())

# Aperçu
display(df.head())

# 5) Mappings libellés (pour affichage lisible plus tard)
rom_lbl = (
    df_raw[["code_rome", "libelle_rome"]]
    .drop_duplicates()
    .set_index("code_rome")["libelle_rome"]
    .to_dict()
)
comp_lbl = (
    df_raw[["code_ogr_competence", "libelle_competence"]]
    .drop_duplicates()
    .set_index("code_ogr_competence")["libelle_competence"]
    .to_dict()
)

Arborescence: Nb de lignes: 38961, Nb de colonnes: 4
Shape df: (38961, 4)
Métiers uniques (brut): 1584
Compétences uniques (brut): 16583
Doublons exacts (4 colonnes): 0

Top sous_cat_comp:
sous_cat_comp
Technique expert    16666
Technique           16507
Transverse           5788
Name: count, dtype: int64

Valeurs coeur_metier après imputation:
coeur_metier
Secondaire    22922
Principale    14768
Émergente      1271
Name: count, dtype: int64


,code_rome,code_ogr_competence,sous_cat_comp,coeur_metier
0,A1101,104016,Technique expert,Secondaire
1,A1101,107141,Technique,Secondaire
2,A1101,110024,Technique,Secondaire
3,A1101,117548,Technique,Secondaire
4,A1101,122567,Technique,Secondaire


___
## 2) Poids et variantes de cellule

Chaque cellule de la matrice **métier × compétence** représente la “force du lien” entre un métier et une compétence.  
Trois stratégies de pondération sont possibles :

- **Binaire**  
  - Valeur = 1 si la compétence est associée au métier, 0 sinon.  
  - Avantages : simple, interprétable, robuste.  
  - Limites : ne tient pas compte de l’importance relative des compétences.

- **Pondéré**  
  - Valeur = poids défini par un barème basé sur `sous_cat_comp` et amplifié par `coeur_metier`.  
  - Exemple de barème :  
    - Technique expert = 5  
    - Principale = 4  
    - Expert = 3  
    - Émergente = 2  
    - Secondaire = 1  
    - Transverse = 0.5  
  - Si `coeur_metier == "Principale"` → on multiplie le poids par 1.2 (boost).  
  - Avantages : injecte la hiérarchie métier/compétence dans les données.  
  - Limites : barème arbitraire → à valider empiriquement.

- **TF-IDF** (inspiré du traitement de texte)  
  - TF = 1 (chaque compétence compte une fois par métier).  
  - IDF = log(N_métiers / nombre_de_métiers_ayant_cette_compétence).  
  - Résultat : les compétences rares (donc discriminantes) ont plus de poids, les compétences trop fréquentes sont atténuées.  
  - Avantages : bonne différenciation des métiers proches.  
  - Limites : peut sous-évaluer des compétences importantes mais très répandues.

L’objectif est de comparer ces trois variantes pour déterminer laquelle donne les meilleures performances de prédiction (Top-1, Top-3).


In [70]:
# Paramètres de pondération
WEIGHTS = {
    "Technique expert": 5.0, 
    "Principale": 4.0, 
    "Expert": 3.0,
    "Émergente": 2.0, "Emergente": 2.0,
    "Secondaire": 1.0,
    "Transverse": 0.5, "Transversal": 0.5,
    "Technique": 2.0
}
COEUR_BOOST_VAL = {"Principale": 1.2}  # sinon 1.0

def add_weights(df_):
    base  = df_["sous_cat_comp"].map(WEIGHTS).fillna(1.0)
    boost = df_["coeur_metier"].map(COEUR_BOOST_VAL).fillna(1.0)
    out = df_.copy()
    out["weight"] = base * boost
    return out

df_w = add_weights(df)

___
## 3) Construction de la matrice sparse + filtrage vocabulaire

L’objectif est de transformer les données en une **matrice sparse (creuse) métiers × compétences**.  
Chaque ligne représente un métier (`code_rome`), chaque colonne une compétence (`code_ogr_competence`), et chaque cellule contient un poids (selon la variante choisie : binaire, pondéré, tfidf).

### Étapes principales
1. **Agrégation**  
   - On regroupe les données par couple `(métier, compétence)` pour éviter les doublons.  
   - On additionne les poids associés.

2. **Construction de la matrice sparse**  
   - Les métiers et compétences sont encodés en indices numériques.  
   - On crée une matrice **CSR** (Compressed Sparse Row) adaptée aux données très creuses.  
   - Exemple : 1584 métiers × 16 583 compétences uniques = une matrice énorme, mais seulement ~39 000 liens réels.

3. **Filtrage du vocabulaire**  
   - **Problème** : beaucoup de compétences sont inutiles pour la prédiction.  
     - Certaines sont trop **rares** (liées à un seul métier).  
     - D’autres sont trop **fréquentes** (présentes dans presque tous les métiers).  
   - Solution : on garde uniquement les compétences utiles avec deux seuils :  
     - `min_df ≥ 2` : au moins 2 métiers différents doivent contenir cette compétence.  
     - `max_df ≤ 80%` : une compétence ne doit pas apparaître dans plus de 80% des métiers.  

4. **Normalisation**  
   - Chaque ligne (métier) est normalisée en norme L2.  
   - Objectif : comparer les profils métiers sur une échelle équivalente (cosinus de similarité).  

### Résultat attendu
- Une matrice compacte et efficace pour la recherche de similarités.  
- Réduction du vocabulaire : on passe de ~16 500 compétences uniques à ~7 000 après filtrage.  
- Gain : suppression du bruit (rares) et des compétences trop génériques.

In [ ]:
# Construction matrice améliorée

def build_matrix(df_w, mode="binaire", min_df=2, max_df_frac=0.80):
    
    """
    Construit une matrice sparse (métiers × compétences).

    Paramètres
    ----------
    df_w : DataFrame
        Doit contenir : code_rome, code_ogr_competence, weight
    mode : str
        "binaire" → présence/absence
        "pondere" → poids conservés
        "tfidf"   → pondération TF-IDF
    min_df : int
        Nombre minimal de métiers devant contenir la compétence (sinon supprimée)
    max_df_frac : float
        Proportion maximale de métiers pouvant contenir la compétence (sinon supprimée)

    Retour
    ------
    X : csr_matrix
        Matrice sparse normalisée (métiers × compétences)
    roms : list
        Liste des codes ROME (ordre des lignes)
    comps : list
        Liste des compétences conservées (ordre des colonnes)
    comp2j : dict
        Mapping {code_ogr_competence → index colonne}
    """

    # 1) Agrégation métier-compétence
    g = df_w.groupby(["code_rome", "code_ogr_competence"], as_index=False)["weight"].sum()

    # 2) Encodage métiers et compétences (trié pour reproductibilité)
    roms  = sorted(g["code_rome"].astype(str).unique())
    comps = sorted(g["code_ogr_competence"].astype(str).unique())
    rom2i = {r: i for i, r in enumerate(roms)}
    comp2j = {c: j for j, c in enumerate(comps)}

    # 3) Construction matrice brute
    rows = g["code_rome"].map(rom2i).to_numpy()
    cols = g["code_ogr_competence"].map(comp2j).to_numpy()
    data = g["weight"].to_numpy()
    X = csr_matrix((data, (rows, cols)), shape=(len(roms), len(comps)))

    # 4) Modes de pondération
    if mode == "binaire":
        X.data[:] = 1.0
    elif mode == "tfidf":
        N = X.shape[0]  # nb métiers
        df_comp = np.diff(X.tocsc().indptr)  # nb métiers par compétence
        idf = np.log((N + 1) / (df_comp + 1)) + 1.0
        X = X.tocoo()
        X.data = idf[X.col]
        X = X.tocsr()
    # "pondere" → garder les poids calculés

    # 5) Filtrage compétences rares / trop fréquentes
    Xc = X.tocsc()
    df_comp = np.diff(Xc.indptr)  # fréquence par compétence
    keep = (df_comp >= min_df) & (df_comp <= X.shape[0] * max_df_frac)

    if keep.any():
        X = X[:, keep]
        comps = [c for c, k in zip(comps, keep) if k]
        comp2j = {c: j for j, c in enumerate(comps)}

    # 6) Normalisation par ligne (comparaison cosinus)
    X = normalize(X, norm="l2", axis=1, copy=False)

    # 7) Contrôle métiers vides
    empty_rows = (X.getnnz(axis=1) == 0).sum()
    if empty_rows > 0:
        print(f"{empty_rows} métiers n'ont plus de compétences après filtrage.")

    return X, roms, comps, comp2j

## Impact du filtrage sur le vocabulaire de compétences

On compare le nombre de compétences uniques avant et après filtrage :

- Avant filtrage : toutes les compétences présentes dans le dataset  
- Après filtrage : uniquement celles présentes dans ≥2 métiers et ≤80% des métiers

In [72]:
# Appel de la fonction pour construire la matrice
X, roms, comps, comp2j = build_matrix(df_w, mode="binaire", min_df=2, max_df_frac=0.80)

# Statistiques de filtrage
nb_comp_avant = df["code_ogr_competence"].nunique()
nb_comp_apres = len(comps)
pct_conserve = 100 * nb_comp_apres / nb_comp_avant
pct_rare = 100 - pct_conserve

print(f"Compétences uniques avant filtrage : {nb_comp_avant}")
print(f"Compétences uniques après filtrage : {nb_comp_apres}")
print(f"% conservées : {pct_conserve:.1f}%")
print(f"% écartées (rares ou trop générales) : {pct_rare:.1f}%")

Compétences uniques avant filtrage : 16583
Compétences uniques après filtrage : 7050
% conservées : 42.5%
% écartées (rares ou trop générales) : 57.5%


___
## 4) Évaluations retrieval

L’objectif est d’évaluer la capacité du modèle à retrouver le bon métier à partir d’un profil de compétences **incomplet**.  
Comme il n’existe pas de dataset séparé `train/test` (chaque métier n’apparaît qu’une fois), on utilise des stratégies de **masquage** pour simuler des cas réels.

### a) Masking (masquage partiel)
- On prend la liste complète de compétences d’un métier.  
- On **cache aléatoirement 60% des compétences** et on ne garde que 40%.  
- On demande au modèle de retrouver le métier d’origine à partir de ce profil réduit.  
- Métriques calculées :  
  - **Top-1 accuracy** : le bon métier est en 1ère position.  
  - **Top-3 accuracy** : le bon métier est dans les 3 premiers résultats.  
- Objectif : simuler un utilisateur qui déclare une partie seulement de ses compétences.

### b) LOSO (Leave-One-Skill-Out)
- Variante extrême : on ne garde **qu’une seule compétence** du métier.  
- On vérifie si le modèle est capable de retrouver le métier correct.  
- Métriques : Top-1 et Top-3.  
- Objectif : mesurer la robustesse minimale, utile comme borne basse.  
- Limite : précision faible attendue, car une seule compétence ne suffit souvent pas à caractériser un métier.

### Pourquoi ces deux évaluations ?
- **Masking** → mesure la robustesse quand il manque de l’information.  
- **LOSO** → mesure la performance quand on a le minimum d’information possible.  
- Ensemble, elles permettent de définir **à partir de combien de compétences** le modèle devient fiable pour une prédiction.

In [ ]:
# Fonctions d'évaluation retrieval

def eval_mask(X, frac_hide=0.60, min_skills=3, seed=42):
    """
    Évalue le modèle en masquant une fraction des compétences de chaque métier.

    Paramètres
    ----------
    X : csr_matrix
        Matrice métiers × compétences (normalisée).
    frac_hide : float
        Proportion des compétences à masquer (ex: 0.60 = garder 40%).
    min_skills : int
        Nombre minimal de compétences nécessaires pour inclure un métier dans l'évaluation.
    seed : int
        Graine aléatoire pour reproductibilité.

    Retour
    ------
    dict : { "Top1": float, "Top3": float, "n": int }
    """
    rng = np.random.default_rng(seed)
    Xr = X.tocsr()
    top1 = top3 = n = 0

    for i in range(X.shape[0]):
        idx = Xr[i].indices
        if len(idx) < min_skills:
            continue
        # Tirage aléatoire d'un sous-ensemble de compétences
        k = max(1, int(len(idx) * frac_hide))
        sub = rng.choice(idx, size=k, replace=False)

        # Construction vecteur requête
        q = csr_matrix((np.ones(k), ([0] * k, sub)), shape=(1, X.shape[1]))
        q = normalize(q, norm="l2", axis=1)

        # Similarité cosinus
        s = (q @ X.T).toarray().ravel()
        order = np.argsort(-s)

        # Scores
        n += 1
        if order[0] == i: 
            top1 += 1
        if i in order[:3]: 
            top3 += 1

    return {"Top1": top1 / n if n else np.nan,
            "Top3": top3 / n if n else np.nan,
            "n": n}


def eval_loso(X, max_per_job=20, seed=42):
    """
    Évalue le modèle en Leave-One-Skill-Out (LOSO).
    Chaque compétence d'un métier est testée seule comme requête.

    Paramètres
    ----------
    X : csr_matrix
        Matrice métiers × compétences (normalisée).
    max_per_job : int
        Nombre max de compétences testées par métier (pour accélérer).
    seed : int
        Graine aléatoire pour reproductibilité.

    Retour
    ------
    dict : { "Top1": float, "Top3": float, "n": int }
    """
    rng = np.random.default_rng(seed)
    Xr = X.tocsr()
    top1 = top3 = n = 0

    for i in range(X.shape[0]):
        idx = Xr[i].indices
        if len(idx) == 0:
            continue
        # Limiter le nombre de tests si métier très riche en compétences
        if len(idx) > max_per_job:
            idx = rng.choice(idx, size=max_per_job, replace=False)

        for j in idx:
            q = csr_matrix(([1.0], ([0], [j])), shape=(1, X.shape[1]))

            # Similarité cosinus
            s = (q @ X.T).toarray().ravel()
            order = np.argsort(-s)

            # Scores
            n += 1
            if order[0] == i: 
                top1 += 1
            if i in order[:3]: 
                top3 += 1

    return {"Top1": top1 / n if n else np.nan,
            "Top3": top3 / n if n else np.nan,
            "n": n}

___
## 5) Run des variantes et tableau comparatif

Une fois la matrice construite, on compare les **trois variantes de pondération** :  
- **Binaire**  
- **Pondéré** (barème basé sur `sous_cat_comp` + boost `coeur_metier`)  
- **TF-IDF**

### Métriques comparées
- **Mask Top-1 / Top-3** : précision quand on cache 60% des compétences.  
- **LOSO Top-1 / Top-3** : précision quand on ne garde qu’une seule compétence.  
- **nnz** : nombre de liens réels dans la matrice (indique la densité).  
- **Vocab** : taille du vocabulaire de compétences conservées après filtrage.  
- **Métiers** : nombre de lignes (devrait rester 1584).

### Objectif de ce tableau
- Identifier si une variante se démarque réellement.  
- Vérifier si la pondération améliore ou dégrade les résultats.  
- Choisir un **mode par défaut** simple et efficace.  

On retient ensuite la variante la plus adaptée pour la suite (profils partiels + inférence).

In [74]:
# Comparaison binaire/pondere/tfidf
df_w = add_weights(df)

rows = []
for mode in ("binaire", "pondere", "tfidf"):
    X, roms, comps, comp2j = build_matrix(df_w, mode=mode, min_df=2, max_df_frac=0.80)
    m = eval_mask(X, 0.60, 3, 42)
    l = eval_loso(X, 20)
    rows.append({
        "Variante": mode,
        "Métiers": X.shape[0],
        "Vocab": X.shape[1],
        "nnz": X.nnz,
        "Mask Top-1": f"{m['Top1']:.3f}",
        "Mask Top-3": f"{m['Top3']:.3f}",
        "LOSO Top-1": f"{l['Top1']:.3f}",
        "LOSO Top-3": f"{l['Top3']:.3f}"
    })

results_df = pd.DataFrame(rows)
display(results_df)

# Résumé textuel en Markdown
best_mask = results_df.loc[results_df["Mask Top-1"].astype(float).idxmax()]
best_loso = results_df.loc[results_df["LOSO Top-1"].astype(float).idxmax()]

from IPython.display import Markdown
md = f"""
### Résumé comparatif des variantes

- Nombre de métiers : **{results_df['Métiers'].iloc[0]}**
- Taille du vocabulaire (après filtrage) : **{results_df['Vocab'].iloc[0]}**
- Nombre de liens (nnz) : ~**{results_df['nnz'].iloc[0]}**

**Masking (60% caché)**
- Meilleure variante : `{best_mask['Variante']}` avec Top-1 = {best_mask['Mask Top-1']} et Top-3 = {best_mask['Mask Top-3']}

**LOSO (1 compétence testée)**
- Meilleure variante : `{best_loso['Variante']}` avec Top-1 = {best_loso['LOSO Top-1']} et Top-3 = {best_loso['LOSO Top-3']}

En pratique, toutes les variantes sont proches. On peut garder **binaire** par défaut (simple et robuste).
"""
display(Markdown(md))


,Variante,Métiers,Vocab,nnz,Mask Top-1,Mask Top-3,LOSO Top-1,LOSO Top-3
0,binaire,1584,7050,29428,0.999,1.000,0.269,0.635
1,pondere,1584,7050,29428,0.994,1.000,0.264,0.628
2,tfidf,1584,7050,29428,0.999,1.000,0.269,0.634



### Résumé comparatif des variantes

- Nombre de métiers : **1584**
- Taille du vocabulaire (après filtrage) : **7050**
- Nombre de liens (nnz) : ~**29428**

**Masking (60% caché)**
- Meilleure variante : `binaire` avec Top-1 = 0.999 et Top-3 = 1.000

**LOSO (1 compétence testée)**
- Meilleure variante : `binaire` avec Top-1 = 0.269 et Top-3 = 0.635

En pratique, toutes les variantes sont proches. On peut garder **binaire** par défaut (simple et robuste).


___
## 6) Profils partiels réalistes

Les tests **Masking** et **LOSO** sont utiles pour explorer les cas extrêmes, mais ils ne reflètent pas toujours la réalité.  
Dans la pratique, un utilisateur renseignera probablement **quelques compétences clés**, pas toutes, mais plus d’une seule.

### Méthode
- Pour chaque métier, on sélectionne aléatoirement **3 ou 5 compétences**.  
- On construit un profil réduit à partir de ces compétences.  
- On demande au modèle de retrouver le métier d’origine.  
- On répète plusieurs fois l’expérience pour fiabiliser les résultats.

### Métriques
- **Top-1 accuracy** : proportion de profils partiels où le métier correct est en 1ère position.  
- **Top-3 accuracy** : proportion de profils partiels où le métier correct est dans les 3 premiers.  

### Objectif
- Mesurer la **robustesse du modèle dans des conditions proches de l’usage réel**.  
- Déterminer **à partir de combien de compétences** le système devient fiable :  
  - Exemple attendu :  
    - 1 compétence → faible précision  
    - 3 compétences → bon niveau de fiabilité  
    - 5 compétences → quasi parfait  

C’est sur la base de ces résultats que l’on fixe les **règles d’usage** (nombre minimal de compétences et seuil de confiance).

In [ ]:
# Évaluation profils partiels réalistes

def eval_partial(X, sizes=(3,5), reps=5, topk=3, seed=42, summarize=True):
    """
    Évalue la robustesse du modèle avec des profils partiels réalistes.
    
    Paramètres
    ----------
    X : csr_matrix
        Matrice métiers × compétences (normalisée).
    sizes : tuple
        Liste des tailles de profils testées (ex: (1,3,5)).
    reps : int
        Nombre de tirages aléatoires par métier et par taille.
    topk : int
        Taille du classement à évaluer (par défaut Top-3).
    seed : int
        Graine aléatoire pour reproductibilité.
    summarize : bool
        Si True, affiche un résumé textuel en Markdown en plus du DataFrame.
    
    Retour
    ------
    DataFrame : k, n, Top1, Topk
    """
    rng = np.random.default_rng(seed)
    Xr = X.tocsr()
    out = []

    for k in sizes:
        top1 = topk_hits = n = 0
        for i in range(X.shape[0]):
            idx = Xr[i].indices
            if len(idx) < k:
                continue
            for _ in range(reps):
                sub = rng.choice(idx, size=k, replace=False)
                q = csr_matrix((np.ones(k), ([0] * k, sub)), shape=(1, X.shape[1]))
                q = normalize(q, norm="l2", axis=1)

                s = (q @ X.T).toarray().ravel()
                order = np.argsort(-s)

                n += 1
                if order[0] == i: 
                    top1 += 1
                if i in order[:topk]: 
                    topk_hits += 1

        out.append({
            "k": k,
            "n": n,
            "Top1": top1 / n if n else np.nan,
            f"Top{topk}": topk_hits / n if n else np.nan
        })

    results = pd.DataFrame(out)

    # Résumé Markdown
    if summarize:
        lines = [f"### Résultats profils partiels (Top-{topk})"]
        for _, row in results.iterrows():
            lines.append(
                f"- Avec **{int(row['k'])} compétences** → "
                f"Top-1 = {row['Top1']:.3f} | Top-{topk} = {row[f'Top{topk}']:.3f}"
            )
        display(Markdown("\n".join(lines)))

    return results

# Exemple d'appel (sur variante binaire)
X, roms, comps, comp2j = build_matrix(df_w, mode="binaire", min_df=2, max_df_frac=0.80)
results_partial = eval_partial(X, sizes=(1,3,5), reps=5, topk=3, seed=42, summarize=True)
display(results_partial)

### Résultats profils partiels (Top-3)
- Avec **1 compétences** → Top-1 = 0.333 | Top-3 = 0.682
- Avec **3 compétences** → Top-1 = 0.925 | Top-3 = 0.992
- Avec **5 compétences** → Top-1 = 0.981 | Top-3 = 0.999

,k,n,Top1,Top3
0,1,7920,0.333460,0.681692
1,3,7880,0.924873,0.992259
2,5,7760,0.980928,0.998711


___
## 7) Inférence simple + seuil de confiance

Une fois la matrice construite et les variantes testées, on a besoin d’une **fonction d’inférence** capable de prédire un métier à partir d’une liste de compétences fournies par un utilisateur.

### Règles d’usage
- **Nombre minimal de compétences** :  
  - En dessous de **3 compétences**, le modèle n’est pas assez fiable.  
  - On retourne donc un statut `needs_more_skills` au lieu d’une prédiction.  

- **Seuil de confiance (cosinus)** :  
  - On calcule un score de similarité cosinus entre le profil utilisateur et chaque métier.  
  - Si le score du Top-1 est **≥ 0.30**, on considère la prédiction comme valide.  
  - Sinon, on renvoie un statut `indecis` et on affiche uniquement le **Top-3** des métiers les plus proches, sans trancher.  
  - La valeur du seuil (0.28–0.32) peut être ajustée en fonction des résultats observés.

### Objectif
- **Éviter les faux positifs** : mieux vaut dire “indécis” que donner un mauvais métier avec confiance.  
- **Proposer une aide à la décision** : même en cas d’indécision, fournir les 3 métiers les plus proches peut guider l’utilisateur.  
- **Maintenir l’interprétabilité** : à terme, il sera possible d’expliquer la prédiction en listant les compétences communes qui justifient le score.

In [76]:
# Inférence + affichage lisible

MIN_SKILLS = 3          # Nombre minimal de compétences pour déclencher une prédiction
THRESHOLD = 0.30        # Seuil de confiance sur le score cosinus
MODE_USED = "binaire"   # Doit matcher la construction de X

def infer_simple(codes_comp, topk=3):
    """
    Prend une liste de codes compétences et retourne les métiers les plus proches
    selon similarité cosinus.

    Applique les règles d’usage :
    - si moins de MIN_SKILLS compétences -> "needs_more_skills"
    - si aucune compétence reconnue -> "no_known_skills"
    - si score_top1 < THRESHOLD -> "indecis" (affiche top-k quand même)
    - sinon -> "ok"
    """
    # Nettoyage des codes compétences
    codes_comp = [str(c).strip() for c in codes_comp]

    # Vérification du minimum
    if len(codes_comp) < MIN_SKILLS:
        return {"status": "needs_more_skills", "min_required": MIN_SKILLS, "topk": []}

    # Mapping des compétences connues
    cols = [comp2j[c] for c in codes_comp if c in comp2j]
    if not cols:
        return {"status": "no_known_skills", "topk": []}

    # Construction du vecteur requête
    q = csr_matrix((np.ones(len(cols)), ([0] * len(cols), cols)), shape=(1, X.shape[1]))
    q = normalize(q, norm="l2", axis=1)

    # Similarité cosinus avec tous les métiers
    scores = (q @ X.T).toarray().ravel()
    order = np.argsort(-scores)[:topk]

    preds = [(roms[i], float(scores[i])) for i in order]
    top1 = preds[0]

    return {
        "status": "ok" if top1[1] >= THRESHOLD else "indecis",
        "threshold": THRESHOLD,
        "top1": top1,
        "topk": preds
    }


def pretty(out):
    """
    Affiche un résultat d'inférence sous forme lisible.
    """
    if out["status"] == "ok":
        code, s = out["top1"]
        print(f"{code} — {rom_lbl.get(code, code)} | score={s:.3f}")
        for code, s in out["topk"]:
            print(f"  - {code} — {rom_lbl.get(code, code)} | {s:.3f}")

    elif out["status"] == "indecis":
        print(f"Indécis (seuil={out['threshold']}). Suggestions :")
        for code, s in out["topk"]:
            print(f"  - {code} — {rom_lbl.get(code, code)} | {s:.3f}")

    elif out["status"] == "needs_more_skills":
        print(f"Ajoute au moins {out['min_required']} compétences.")

    elif out["status"] == "no_known_skills":
        print("Aucune compétence reconnue.")

___
## 8) Démo d’inférence

Pour vérifier que tout le pipeline fonctionne, on fait une **requête test** en donnant quelques compétences.  
L’objectif est de voir :  
- si le système renvoie bien un métier avec son libellé,  
- le score associé (cosinus de similarité),  
- et les métiers alternatifs dans le Top-3 si disponibles.

### Points attendus
- Si l’utilisateur fournit au moins **3 compétences** et que le score du Top-1 dépasse **0.30**, la prédiction est acceptée (`status = ok`).  
- Si le score est inférieur au seuil, le système renvoie `indecis` avec le **Top-3** des métiers les plus proches.  
- Si moins de 3 compétences sont données, le système renvoie `needs_more_skills`.  
- Les résultats doivent afficher à la fois les **codes ROME** et les **libellés métiers** pour être interprétables.

Cette démo permet de valider le bon fonctionnement de la fonction `infer_simple` et de s’assurer que les règles d’usage (min compétences, seuil de confiance) sont correctement appliquées.

In [77]:
# Démonstration

def demo_request(req):
    """
    Exécute une démo d'inférence à partir d'une liste de compétences
    et affiche les résultats avec les libellés.
    """
    # Affichage clair de la requête
    print("\n Requête compétences :")
    for c in req:
        print(f"  - {c} — {comp_lbl.get(c, c)}")

    # Inférence
    out = infer_simple(req)
    print("\n Résultat :")
    pretty(out)


# --- Démo 1 : Requête valide avec 3 compétences connues ---
req1 = ["110024", "404006", "122730"]
demo_request(req1)

# --- Démo 2 : Requête trop courte (< MIN_SKILLS) ---
req2 = ["110024"]
demo_request(req2)

# --- Démo 3 : Compétence inconnue ---
req3 = ["999999", "888888", "777777"]
demo_request(req3)


 Requête compétences :
  - 110024 — 110024
  - 404006 — 404006
  - 122730 — 122730

 Résultat :
A1101 — Conducteur / Conductrice d'engins agricoles | score=0.408
  - A1101 — Conducteur / Conductrice d'engins agricoles | 0.408
  - A1419 — Ouvrier agricole polyvalent / Ouvrière agricole polyvalente | 0.272
  - N4111 — Conducteur / Conductrice super poids lourd de l'armée | 0.160

 Requête compétences :
  - 110024 — 110024

 Résultat :
Ajoute au moins 3 compétences.

 Requête compétences :
  - 999999 — 999999
  - 888888 — 888888
  - 777777 — 777777

 Résultat :
Aucune compétence reconnue.


___
## 9) Sauvegarde et rechargement des artefacts - Interopérabilité

Une fois le modèle validé, il est important de sauvegarder les objets nécessaires à l’inférence afin de ne pas reconstruire toute la pipeline à chaque fois.

### Artefacts sauvegardés
- **X** : la matrice sparse normalisée (métiers × compétences).  
- **roms** : la liste des métiers (ordre des lignes de X).  
- **comp2j** : le dictionnaire {code_compétence → index de colonne dans X}.  
- **rom_lbl** : mapping {code_rome → libellé métier}.  
- **comp_lbl** : mapping {code_compétence → libellé compétence}.  

Ces objets permettent de reproduire exactement le processus de vectorisation et d’interprétation lors de l’inférence.

### Objectif
- Faciliter l’intégration dans une API ou un outil d’analyse.  
- Éviter de refaire le prétraitement et le calcul de la matrice à chaque utilisation.  
- Garantir la cohérence entre les phases d’entraînement/évaluation et la mise en production.

### Rechargement
On peut ensuite recharger ces artefacts avec `joblib.load()` et utiliser directement la fonction `infer_simple` pour prédire un métier à partir d’une liste de compétences.

In [ ]:
# Reload complet + harmonisation + sanity + démo

joblibfile = "metiers_comp.joblib"

# 0) Sauvegarde des artefacts si besoin
if not os.path.exists(joblibfile):
    joblib.dump({
        "X": X,
        "roms": roms,
        "comp2j": comp2j,
        "rom_lbl": rom_lbl,
        "comp_lbl": comp_lbl
    }, joblibfile)
 
# 1) Reload artefacts
bundle = joblib.load(joblibfile)
X       = bundle["X"]
roms    = bundle["roms"]
comp2j  = bundle["comp2j"]
rom_lbl = bundle.get("rom_lbl", {})
comp_lbl= bundle.get("comp_lbl", {})

# 2) Harmoniser les types de clés (tout en str)
roms     = [str(r).strip() for r in roms]
rom_lbl  = {str(k).strip(): v for k, v in rom_lbl.items()}
comp2j   = {str(k).strip(): v for k, v in comp2j.items()}
comp_lbl = {str(k).strip(): v for k, v in comp_lbl.items()}

# 3) Sanity check
print("Artefacts rechargés\n"
      f"- Matrice : {X.shape[0]} métiers × {X.shape[1]} compétences | nnz={X.nnz}\n"
      f"- Métiers  : {len(roms)}\n"
      f"- Compétences (vocab) : {len(comp2j)}")

# Exemple contrôle
sample_rom  = roms[0]
sample_comp = next(iter(comp2j.keys()))
print("\nExemple contrôle :")
print(f"  Métier {sample_rom} → {rom_lbl.get(sample_rom, '?')}")
print(f"  Compétence {sample_comp} → {comp_lbl.get(str(sample_comp), '?')}")

# 4) Inférence simple (mêmes règles que dans le notebook)
MIN_SKILLS = 3
THRESHOLD  = 0.30

def infer_simple(codes_comp, topk=3):
    """Retourne métiers les plus proches (cosinus). Règles: ≥3 skills, seuil 0.30."""
    codes_comp = [str(c).strip() for c in codes_comp]
    if len(codes_comp) < MIN_SKILLS:
        return {"status":"needs_more_skills","min_required":MIN_SKILLS,"topk":[]}
    cols = [comp2j[c] for c in codes_comp if c in comp2j]
    if not cols:
        return {"status":"no_known_skills","topk":[]}
    q = csr_matrix((np.ones(len(cols)), ([0]*len(cols), cols)), shape=(1, X.shape[1]))
    q = normalize(q, norm="l2", axis=1)
    s = (q @ X.T).toarray().ravel()
    order = np.argsort(-s)[:topk]
    preds = [(roms[i], float(s[i])) for i in order]
    top1 = preds[0]
    return {"status":"ok" if top1[1] >= THRESHOLD else "indecis",
            "threshold":THRESHOLD,"top1":top1,"topk":preds}

def pretty(out):
    if out["status"] == "ok":
        code, sc = out["top1"]
        print(f"\n{code} — {rom_lbl.get(code, code)} | score={sc:.3f}")
        for code, s in out["topk"]:
            print(f"  - {code} — {rom_lbl.get(code, code)} | {s:.3f}")
    elif out["status"] == "indecis":
        print(f"\nIndécis (seuil={out['threshold']}). Suggestions:")
        for code, s in out["topk"]:
            print(f"  - {code} — {rom_lbl.get(code, code)} | {s:.3f}")
    elif out["status"] == "needs_more_skills":
        print(f"\nAjoute au moins {out['min_required']} compétences.")
    elif out["status"] == "no_known_skills":
        print("\nAucune compétence reconnue.")

# 5) Démo
def demo_request(req):
    print("\nRequête compétences :")
    for c in req:
        print(f"  - {str(c).strip()} — {comp_lbl.get(str(c).strip(), str(c).strip())}")
    out = infer_simple(req, topk=3)
    pretty(out)

# Exemple
demo_request(["110024","404006","122730"])

Artefacts rechargés
- Matrice : 1584 métiers × 7050 compétences | nnz=29428
- Métiers  : 1584
- Compétences (vocab) : 7050

Exemple contrôle :
  Métier A1101 → Conducteur / Conductrice d'engins agricoles
  Compétence 100023 → Abattre un arbre

Requête compétences :
  - 110024 — Conduire un poids lourd
  - 404006 — Conduire un véhicule agricole
  - 122730 — Préparer les sols, les plantations (épandage, semis, etc.)

A1101 — Conducteur / Conductrice d'engins agricoles | score=0.408
  - A1101 — Conducteur / Conductrice d'engins agricoles | 0.408
  - A1419 — Ouvrier agricole polyvalent / Ouvrière agricole polyvalente | 0.272
  - N4111 — Conducteur / Conductrice super poids lourd de l'armée | 0.160


___
# Synthèse et règles d’usage

## Résultats principaux
- **Matrice finale** : 1584 métiers × 7050 compétences | 29 428 liens réels  
  - On est passé de 16 583 compétences uniques initialement à 7050 après filtrage.  
  - Les compétences trop **rares** (présentes dans <2 métiers) et trop **générales** (>80% des métiers) ont été écartées.  
  - Résultat : un vocabulaire plus compact et plus discriminant.
- **Comparaison des variantes** :
  - *Binaire* = aussi performant que *pondere* ou *tfidf*
  - **Choix par défaut** : **binaire** (simple, robuste)
- **Profils partiels** :
  - 1 compétence → Top-1 = 0.34 | Top-3 = 0.68 (faible, non fiable seul)
  - 3 compétences → Top-1 = 0.92 | Top-3 = 0.99 (**excellent**)
  - 5 compétences → Top-1 = 0.98 | Top-3 ≈ 1.0 (**quasi parfait**)
- **Inférence test** :
  - Exemple avec 3 compétences → prédiction correcte (`A1101`) avec score = 0.408
  - Suggestions Top-3 cohérentes et interprétables


## Règles d’usage
- **Nombre minimal de compétences** : 3  
- **Seuil de confiance cosinus** : 0.30  
  - Si `score_top1 ≥ 0.30` → prédiction acceptée  
  - Sinon → statut *indécis*, afficher le **Top-3** uniquement  
- **Mode par défaut** : binaire  
- **Explication** : afficher les compétences pivot qui soutiennent la prédiction


## Prochaines étapes possibles
- Intégrer l’**explication automatique** (liste des compétences communes) pour chaque prédiction
- Créer une **API ou interface** simple autour de la fonction `infer_simple`
- (Optionnel) Tester d’autres colonnes (`enjeu`, `objectif`, `domaine_compétence`) si besoin d’enrichir les signaux